NLP - or Natural Language Processing ou processamento de linguagem natural é de uma forma simples é um conjunto amplo de técnica desenvolvidas para ajudar as máquinas a aprender textos. A NLP da vida de chatbots a motores de busca e é usado para diversas tarefas como análise de sentimentos e tradução. 


In [12]:
#IMPORTANDO LIBS
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing


In [13]:
# IMPORTANDO DADOS
train_df = pd.read_csv("input/nlp-getting-start/train.csv")
test_df = pd.read_csv("input/nlp-getting-start/test.csv")


In [14]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


Exemplo de tweets normais

In [15]:
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [16]:
train_df[train_df["target"] == 0]["text"].values[0:5]

array(["What's up man?", 'I love fruits', 'Summer is lovely',
       'My car is so fast', 'What a goooooooaaaaaal!!!!!!'], dtype=object)

Exemplo de um tweet de desastre:

In [17]:
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

## Construindo vetores
A teoria por trás deste modelo é bem simples: as palavras contendo cada tweet são bons indicadores se eles são reais desastres ou não. (Não está inteiramente correto mas é um bom lugar pra começar)

Para isso podemos usar a biblioteca CountVectorizer do scikit-learn para contar as palavras em cada tweet e transformá-las em dados nosso modelo de machine learning possa ser processado. 

O vetor é nesse contexto, é um conjunto de numeros que o modelo de machine learning pode trabalhar. 



In [18]:
count_vectorizer = feature_extraction.text.CountVectorizer()

## let's get counts for the first 5 tweets in the data
example_train_vectors = count_vectorizer.fit_transform(train_df["text"][0:5])

In [19]:
print(example_train_vectors)

  (0, 34)	1
  (0, 12)	1
  (0, 5)	1
  (0, 49)	1
  (0, 39)	1
  (0, 29)	1
  (0, 50)	1
  (0, 13)	1
  (0, 25)	1
  (0, 4)	1
  (0, 18)	1
  (0, 52)	1
  (0, 3)	1
  (1, 17)	1
  (1, 16)	1
  (1, 26)	1
  (1, 24)	1
  (1, 42)	1
  (1, 44)	1
  (1, 11)	1
  (2, 5)	2
  (2, 3)	1
  (2, 41)	1
  (2, 7)	1
  (2, 51)	1
  :	:
  (2, 32)	1
  (2, 15)	1
  (3, 21)	1
  (3, 14)	1
  (3, 32)	1
  (3, 1)	1
  (3, 0)	1
  (3, 35)	1
  (3, 40)	1
  (3, 53)	1
  (3, 10)	1
  (4, 50)	1
  (4, 53)	1
  (4, 23)	1
  (4, 20)	1
  (4, 46)	1
  (4, 36)	1
  (4, 19)	2
  (4, 43)	1
  (4, 2)	1
  (4, 6)	1
  (4, 48)	1
  (4, 38)	1
  (4, 22)	1
  (4, 45)	1


In [20]:
## usar o .todense() pois os vetores são  enxutos apenas elementos diferente de zero são salvos.
print(example_train_vectors[0].todense().shape)
print(example_train_vectors[0].todense())

(1, 54)
[[0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
  0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0]]


A informação acima mostra que, há 54 palavras únicas (ou tokens) nos primeiros 5 tweets. 

O primeiro tweet contem apenas alguns desses tokens únicos, todos os contadores que não são zero acima são tokes que existem no primeiro tweet. Agora vamos criar um vetor para todos os tweets.


In [21]:
train_vectors = count_vectorizer.fit_transform(train_df["text"])

## note that we're NOT using .fit_transform() here. Using just .transform() makes sure
# that the tokens in the train vectors are the only ones mapped to the test vectors - 
# i.e. that the train and test vectors use the same set of tokens.
test_vectors = count_vectorizer.transform(test_df["text"])

## Nosso modelo

Como mencionado antes, podemos pensar nas palavras contidas em cada tweet são bons indicadores se eles são sobre desastres reais ou não. A presença de uma palavra em particular (ou o conjunto de palavras) em um tweet deve estar relacionado diretamente se o tweet é real ou não. 

O que estamos assumindo aqui é uma correlação linear, então vamos construir um modelo linear e ver.



In [22]:
## Our vectors are really big, so we want to push our model's weights
## toward 0 without completely discounting different words - ridge regression 
## is a good way to do this.
clf = linear_model.RidgeClassifier()

Vamos testar nosso modelo e ver quão bem ele está em relação aos dados de treino. Para isso, vamos usar a cross-validation, onde treinamos parte dos dados conhecidos e validamos com o restante. Se fizermos isso algumas vezes (com porções diferentes) podemos ter uma boa ideia de como um modelo particular ou método performa. 

A métrica para esta competição é F1, então vamos usá-la aqui.



In [23]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores


array([0.59421842, 0.56498283, 0.64113893])

Os scores acimas não são terríveis! Parecessem que assumimos um score próximo a 0.65 no leaderboard. Existem várias formas de potencializar e melhorar isso (TFIDF, LSA, LSTM / RNNs, ..) - Faça um teste!

Neste meio tempo, faremos previsões no nosso conjunto de treino e vamos construir dados de envio para a competição. 


In [24]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [25]:
sample_submission = pd.read_csv("input/nlp-getting-start/sample_submission.csv")

In [26]:
sample_submission["target"] = clf.predict(test_vectors)


In [27]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [28]:
sample_submission.to_csv("submission.csv", index=False)

## TFIDF
Tentar melhorar os dados usando TF-IDF

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorized = TfidfVectorizer()
train_vector_tfid = vectorized.fit_transform(train_df["text"])

scores = model_selection.cross_val_score(clf, train_vector_tfid, train_df["target"], cv=3, scoring="f1")
scores

sample_submission = pd.read_csv("input/nlp-getting-start/sample_submission.csv")
sample_submission["target"] = clf.predict(test_vectors)
sample_submission.to_csv("output/nlp-getting-start/submission.csv", index=False)


Aplicando o tfid, que é feito para trabalhar NPL já ficou melhor. Uma boa explicação desse método encontrei no FCC: https://www.freecodecamp.org/news/how-to-process-textual-data-using-tf-idf-in-python-cd2bbc0a94a3/

## LSA

Latent Semantic Analysis, sumarização automática de textos. Identifica, coleta ou gera sentenças relevantes a partir de documentos textuais, produzindo ao final um sumário.  Sumários caracterizam-se por serem construídos de modo a não perderem consideravelmente o significado original, apesar de conterem menos informações e poderem apresentar diferentes estruturas, em relação a suas fontes. Métodos que coletam palavras e sentenças diretamente do texto e as agrupam no resumo sem preocupação com reescrita são chamados de extrativos. Já métodos que usam uma representação linguística interna para gerar resumos mais humanos, parafraseando a intenção do texto original, são chamados de abstratos. Além de considerar as palavras mais frequentes, tratava três componentes adicionais: palavras pragmáticas (cue words); palavras de título e título; e indicadores estruturais (localização da sentença no texto). O sistema de Edmundson era parametrizado de acordo com a influência de todas estas heurísticas, atribuindo uma pontuação (positiva ou negativa) à sentença. Depois que todas as heurísticas eram aplicadas, as frases de maior pontuação eram incluídas no resumo.

Outra técnica usada em sumarização automática de textos chama-se LSA – Latent Semantic Analysis. O LSA analisa documentos para encontrar os significados ou conceitos subjacentes. É um método que se originou em outra área de processamento de linguagem natural, chamada recuperação de informações, na qual palavras são informadas como argumentos para a recuperação de textos.

O LSA parte do pressuposto de que uma palavra descreve um conceito e que os autores dos textos escolhem palavras para representar estes conceitos. Selecionar as sentenças de um texto significa, portanto, selecionar os conceitos mais importantes, através das palavras que os representam. O algoritmo do LSA identifica quais palavras são usadas em conjunto e quais palavras comuns são vistas em frases diferentes. Uma grande quantidade de palavras comuns entre frases diferentes indica que as frases são semanticamente relacionadas.

Existem técnicas que utilizam grafos para representação das sentenças. É o caso do TextRank, um algoritmo não supervisionado, baseado em grafos, usado para sumarização automática de textos, que também pode ser usado ​​para obter as palavras-chave de um documento. Foi introduzido por Rada Mihalcea e Paul Tarau [6] a partir de uma variação do algoritmo PageRank.

Este modelo de classificação baseia-se na ideia de votação ou recomendação. Pode-se imaginar que quando um vértice aponta para outro, é basicamente um voto que esta sendo dado para este outro vértice. Assim, quanto maior o número de votos que o vértice recebe, maior é a importância que ele tem. Além disso, o algoritmo também considera a importância do vértice do qual se está recebendo o voto. Se o vértice que vota também recebe muitos votos, ele tem importância maior do que outro vértice que recebe poucos votos. Os vértices podem representar palavras, n-Gramas ou sentenças completas.

## LSA em Python

A bilbioteca em python se chama Sumy e abaixo segue o código de como utilizá-la no exemplo anterior.

In [ ]:
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

for i in range(len(train_df["text"])):
    parser = PlaintextParser.from_string(train_df["text"][i],Tokenizer("english"))
    summarizer_lsa = LsaSummarizer()
    summary_2 =summarizer_lsa(parser.document,2)

    for sentence in summary_2:
        print(sentence)
    


Problemas ao instalar o sumy, codigo acima não funcionou.

Criar um pré-processamento de texto para verificar se consigo melhorar um pouco mais os scores:

In [ ]:
import nltk
nltk.download()

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'[a-z]+')
stop_words = set(stopwords.words('english'))

def preprocess(document):
    document = document.lower() # Convert to lowercase
    words = tokenizer.tokenize(document) # Tokenize
    words = [w for w in words if not w in stop_words] # Removing stopwords
    # Lemmatizing
    for pos in [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]:
        words = [wordnet_lemmatizer.lemmatize(x, pos) for x in words]
    return " ".join(words)

print(train_df['text'].apply(preprocess))
